### Using cellpose for deep-learning powered segmentation of lattice lightsheet data
Multichannel data

Custom cellpose module: `cellpose_2.py`
Within napari-lattice, the `config module` can be used to access the currently active channel (`config.channel`) and time (`config.time`). This is particularly useful when you want to batch process your data. In example below, we will apply a different model based on the active channel

In [1]:
#The function is saved as `cellpose_2.py` and should be in the same folder with the workflow file generated below inorder to function in napari-lattice


### cellpose_2.py
import numpy as np
from cellpose import models
from napari_lattice import config


def predict_cellpose_multich(img,model_channel1:str,model_channel2:str):
    #if first channel, use model specified in model_channel1
    if config.channel == 0:
        model_type = model_channel1
    #if second channel, use model specified in model_channel2
    elif config.channel == 1:
        model_type = model_channel2
    model = models.Cellpose(gpu=True, model_type=model_type)
    channels = [0,0]
    img =np.array(img)
    masks, flows, styles, diams = model.eval(img, flow_threshold=None, channels=channels, diameter=25, do_3D=True)
    return masks


c:\Users\Pradeep\.conda\envs\lattice_cellpose\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define the cellpose workflow, where we apply the cellpose models `nuclei` to channel1 and `cyto2` to channel2. This is just to demonstrate how we can apply channel-specific operations.

In [2]:
from napari_workflows import Workflow
from cellpose_2 import predict_cellpose_multich

#We initialise a workflow
cellpose_multi_ch_workflow = Workflow()

#define cellpose prediction

input_arg = "input"
task_name = "cellpose_multich"
cellpose_multi_ch_workflow.set(task_name,predict_cellpose_multich,input_arg,model_channel1="nuclei",model_channel2="cyto2")
#Printing a workflow will show you the arguments
print(cellpose_multi_ch_workflow)

Workflow:
cellpose_multich <- (<function predict_cellpose_multich at 0x00000283780A7DC0>, 'input', 'nuclei', 'cyto2')



In [3]:
from napari_workflows import _io_yaml_v1

_io_yaml_v1.save_workflow("cellpose_2_workflow.yml",cellpose_multi_ch_workflow)

In future versions of `napari-lattice`, we will add support to pass multi-channel images for use in Cellpose

#TODO: ADD EXAMPLE